In [1]:
from IPython.display import clear_output
import time
import concurrent.futures 
from tqdm import tqdm
import logging
logging.basicConfig(filename='logs.log', level=logging.INFO, datefmt="%H:%M:%S")
import pandas as pd
import warnings 
import numpy as np
import math
from scipy import stats
import os
os.environ['NUMEXPR_MAX_THREADS'] = '11'
os.environ['NUMEXPR_NUM_THREADS'] = '11'
import numexpr as ne

#wyciszenie powiadomień
warnings.filterwarnings("ignore", category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

#sklearn i patch
from sklearnex import patch_sklearn
patch_sklearn() #optymalizacja biblioteki sklearn

import sklearn
#preprocesing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.impute import KNNImputer, SimpleImputer

#modele
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor

#metryki
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

#SMOTE
from imblearn.over_sampling import SMOTE

#inne
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

#wykresy
import matplotlib.pyplot as plt
import seaborn as sns

t0=time.time()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
df=pd.read_parquet('data/ads/aio_scraped_data_concated_dataframe_preprocessed_2024_3_31_9_16.parquet',dtype_backend='pyarrow').drop('id',axis=1)

In [3]:
df.liczba_pokoi

0        3
1        3
2        2
3        3
4        2
        ..
99136    2
99137    2
99138    4
99139    2
99140    2
Name: liczba_pokoi, Length: 99141, dtype: string[pyarrow]

In [4]:
[(col,df[col].isna().sum()) for col in df.columns if df[col].isna().sum()>0]

[('powierzchnia', 4),
 ('forma_wlasnosci', 24602),
 ('liczba_pokoi', 80),
 ('stan_wykonczenia', 21741),
 ('czynsz', 54548),
 ('miejsce_parkingowe', 42618),
 ('ogrzewanie', 18235),
 ('dostepne_od', 79411),
 ('rok_budowy', 24353),
 ('rodzaj_zabudowy', 15912),
 ('okna', 36564),
 ('material_budynku', 46051),
 ('ktore_pietro', 4515),
 ('ile_pieter', 10435)]

In [5]:
df=df.dropna(subset=['powierzchnia','ktore_pietro'])

In [6]:
[col for col in df.columns if df[col].isna().sum()>0]

['forma_wlasnosci',
 'liczba_pokoi',
 'stan_wykonczenia',
 'czynsz',
 'miejsce_parkingowe',
 'ogrzewanie',
 'dostepne_od',
 'rok_budowy',
 'rodzaj_zabudowy',
 'okna',
 'material_budynku',
 'ile_pieter']

In [7]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
powierzchnia,94623.0,<NA>,<NA>,<NA>,59.503358,47.908353,1.0,42.619999,54.0,67.660004,5074.0
forma_wlasnosci,71665,4,pełna własność,67470,NaN,NaN,NaN,NaN,NaN,NaN,NaN
liczba_pokoi,94544,11,2,36468,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stan_wykonczenia,74528,3,do zamieszkania,39863,NaN,NaN,NaN,NaN,NaN,NaN,NaN
czynsz,43190,2006,500,2993,NaN,NaN,NaN,NaN,NaN,NaN,NaN
miejsce_parkingowe,53784,1,garaż/miejsce parkingowe,53784,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ogrzewanie,77823,6,miejskie,53870,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rynek,94623,2,wtórny,60982,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dostepne_od,18769,919,2024-03-01,551,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rok_budowy,71570,242,2023,11579,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94623 entries, 0 to 99140
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype          
---  ------                       --------------  -----          
 0   powierzchnia                 94623 non-null  float[pyarrow] 
 1   forma_wlasnosci              71665 non-null  string[pyarrow]
 2   liczba_pokoi                 94544 non-null  string[pyarrow]
 3   stan_wykonczenia             74528 non-null  string[pyarrow]
 4   czynsz                       43190 non-null  string[pyarrow]
 5   miejsce_parkingowe           53784 non-null  string[pyarrow]
 6   ogrzewanie                   77823 non-null  string[pyarrow]
 7   rynek                        94623 non-null  string[pyarrow]
 8   dostepne_od                  18769 non-null  string[pyarrow]
 9   rok_budowy                   71570 non-null  string[pyarrow]
 10  rodzaj_zabudowy              80215 non-null  string[pyarrow]
 11  okna                         6026

In [9]:
df=df.drop(['adres','ad_photo','price_currency'],axis=1)

In [10]:
categor=['forma_wlasnosci','stan_wykonczenia','miejsce_parkingowe','ogrzewanie','rynek','rodzaj_zabudowy','okna','winda','material_budynku','poster_type','region_id','subregion_id']
df[categor]=df[categor].astype('category')
df['liczba_pokoi']=df['liczba_pokoi'].astype('Uint64[pyarrow]')
df['ktore_pietro']=df['ktore_pietro'].astype('int64[pyarrow]')
df['ile_pieter']=df['ile_pieter'].astype('int64[pyarrow]')
df['czynsz']=df['czynsz'].str.replace(',','.')
df['czynsz']=df['czynsz'].str.replace(' ','').astype('float[pyarrow]')
df['rok_budowy']=df['rok_budowy'].astype('int64[pyarrow]')
df['dostepne_od']=df['dostepne_od'].astype('date64[pyarrow]')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94623 entries, 0 to 99140
Data columns (total 48 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   powierzchnia                 94623 non-null  float[pyarrow]     
 1   forma_wlasnosci              71665 non-null  category           
 2   liczba_pokoi                 94544 non-null  uint64[pyarrow]    
 3   stan_wykonczenia             74528 non-null  category           
 4   czynsz                       43190 non-null  float[pyarrow]     
 5   miejsce_parkingowe           53784 non-null  category           
 6   ogrzewanie                   77823 non-null  category           
 7   rynek                        94623 non-null  category           
 8   dostepne_od                  18769 non-null  date64[ms][pyarrow]
 9   rok_budowy                   71570 non-null  int64[pyarrow]     
 10  rodzaj_zabudowy              80215 non-null  catego

In [12]:
def out_outliers_and_bad_data(df_copy,byZScore,byIQR,iter,cols):
    for _ in range(iter):
        for col in cols:
            if byIQR==True: #usunięcie outlierów za pomocą IQR
                Q3 = df_copy[col].quantile(q=.75)
                Q1 = df_copy[col].quantile(q=.25)
                IQR = stats.iqr(df_copy[col])
                df_copy=df_copy[((df_copy[col] > (Q1-1.5*IQR)) and (df_copy[col] < (Q3+1.5*IQR)))]
            if byZScore==True: #to samo ale za pomocą z-score
                df_copy=df_copy[(np.abs(stats.zscore(df_copy[col])) < 3)]
    return df_copy

def dropCorr(df_copy,corrRatio): # usuwa kolumny dla których korelacja jest większa niż corrRatio
    corr_matrix = df_copy.corr(numeric_only=True).abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > corrRatio)]
    df_copy=df_copy.drop(to_drop, axis=1)
    return df_copy

In [13]:
[(col,df[col].isna().sum()) for col in df.columns if df[col].isna().sum()>0]

[('forma_wlasnosci', 22958),
 ('liczba_pokoi', 79),
 ('stan_wykonczenia', 20095),
 ('czynsz', 51433),
 ('miejsce_parkingowe', 40839),
 ('ogrzewanie', 16800),
 ('dostepne_od', 75854),
 ('rok_budowy', 23053),
 ('rodzaj_zabudowy', 14408),
 ('okna', 34354),
 ('material_budynku', 43371),
 ('ile_pieter', 6902)]

In [14]:
for col in [col for col in df.columns if df[col].isna().sum()>0]:
    print(col,{cat:number for number,cat in enumerate(df[col].unique())})

forma_wlasnosci {'pełna własność': 0, <NA>: 1, 'spółdzielcze wł. prawo do lokalu': 2, 'udział': 3, 'użytkowanie wieczyste / dzierżawa': 4}
liczba_pokoi {3: 0, 2: 1, 4: 2, 5: 3, 1: 4, 11: 5, 9: 6, 6: 7, 8: 8, 7: 9, 10: 10, <NA>: 11}
stan_wykonczenia {'do zamieszkania': 0, <NA>: 1, 'do wykończenia': 2, 'do remontu': 3}
czynsz {980.0: 0, <NA>: 1, 600.0: 2, 500.0: 3, 1600.0: 4, 1350.0: 5, 700.0: 6, 720.0: 7, 350.0: 8, 252.0: 9, 560.0: 10, 730.0: 11, 650.0: 12, 850.0: 13, 235.83999633789062: 14, 930.0: 15, 454.0: 16, 300.0: 17, 550.0: 18, 450.0: 19, 1100.0: 20, 455.0: 21, 380.0: 22, 10.0: 23, 1050.0: 24, 1500.0: 25, 750.0: 26, 823.0: 27, 400.0: 28, 660.0: 29, 8.0: 30, 420.0: 31, 800.0: 32, 315.0: 33, 598.0: 34, 219.0: 35, 780.0: 36, 1000.0: 37, 580.0: 38, 335.0: 39, 900.0: 40, 630.0: 41, 480.0: 42, 430.0: 43, 460.0: 44, 484.0: 45, 1400.0: 46, 490.0: 47, 972.0: 48, 250.0: 49, 790.0: 50, 970.0: 51, 330.0: 52, 685.3099975585938: 53, 990.0: 54, 1200.0: 55, 3.0: 56, 0.009999999776482582: 57, 555

In [15]:

row_nan_count_treshold=5
#df=df.dropna(thresh=col_nan_count_treshold,axis=1)
df=df.dropna(thresh=row_nan_count_treshold,axis=0)

if True:    
    for col in set([col for col in df.columns if df[col].isna().sum()>0])-set(['dostepne_od','miejsce_parkingowe']):
        print(col)
        if df[col].dtype == 'string[pyarrow]':
            df[col] =df[col].fillna('to_fill')
            mapping={cat:number for number,cat in enumerate(df[col].unique())}
            inversed_mapping = {v: k for k, v in mapping.items()}
            df[col]=df[col].map(mapping)
            imputer = KNNImputer(weights="distance", missing_values=mapping['to_fill'])
            df[[col,'latitude','longitude','ad_price']]=pd.DataFrame(imputer.fit_transform(df[[col,'latitude','longitude','ad_price']]),columns=[col,'latitude','longitude','ad_price'],index=df.index)
            df[col]=df[col].astype('int64')
            #df[col]=df[col].map(inversed_mapping)
        elif df[col].dtype == 'category':
            df[col]=df[col].cat.add_categories('to_fill')
            df[col] =df[col].fillna('to_fill')
            mapping={cat:number for number,cat in enumerate(df[col].unique())}
            inversed_mapping = {v: k for k, v in mapping.items()}
            df[col]=df[col].map(mapping)
            imputer = KNNImputer(weights="distance", missing_values=mapping['to_fill'])
            df[[col,'latitude','longitude','ad_price']]=pd.DataFrame(imputer.fit_transform(df[[col,'latitude','longitude','ad_price']]),columns=[col,'latitude','longitude','ad_price'],index=df.index)
            df[col]=df[col].astype('int64')
            #df[col]=df[col].map(inversed_mapping)
        elif df[col].dtype != 'string[pyarrow]':
            df[col] =df[col].fillna(404)
            mapping={cat:number for number,cat in enumerate(df[col].unique())}
            inversed_mapping = {v: k for k, v in mapping.items()}
            df[col]=df[col].map(mapping)
            imputer = KNNImputer(weights="distance", missing_values=mapping[404])
            df[[col,'latitude','longitude','ad_price']]=pd.DataFrame(imputer.fit_transform(df[[col,'latitude','longitude','ad_price']]),columns=[col,'latitude','longitude','ad_price'],index=df.index)
            df[col]=df[col].astype('int64')
            #df[col]=df[col].map(inversed_mapping)
df


forma_wlasnosci


material_budynku
okna
ogrzewanie
rok_budowy
czynsz
ile_pieter
liczba_pokoi
stan_wykonczenia
rodzaj_zabudowy


,powierzchnia,forma_wlasnosci,liczba_pokoi,stan_wykonczenia,czynsz,miejsce_parkingowe,ogrzewanie,rynek,dostepne_od,rok_budowy,...,meble,piekarnik,pralka,telewizor,zmywarka,dwupoziomowe,klimatyzacja,oddzielna_kuchnia,piwnica,pom__uzytkowe
0,65.839996,0,0,0,0,garaż/miejsce parkingowe,0,wtórny,<NA>,0,...,0,0,0,0,0,0,0,1,1,1
1,64.300003,0,0,0,288,garaż/miejsce parkingowe,0,wtórny,2023-11-25,1,...,0,1,1,0,1,0,0,0,1,0
3,48.000000,0,0,0,2,garaż/miejsce parkingowe,0,wtórny,<NA>,2,...,0,0,0,0,0,0,0,0,1,0
4,49.349998,0,1,2,3,garaż/miejsce parkingowe,0,wtórny,2024-03-05,41,...,0,0,0,0,0,0,0,1,0,0
5,93.500000,0,2,2,86,garaż/miejsce parkingowe,1,pierwotny,<NA>,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99136,36.400002,2,1,0,42,<NA>,0,wtórny,<NA>,24,...,1,0,1,0,0,0,0,0,1,0
99137,37.000000,2,1,1,47,<NA>,0,wtórny,<NA>,43,...,0,0,0,0,0,0,0,0,1,0
99138,55.500000,0,2,3,17,garaż/miejsce parkingowe,0,wtórny,<NA>,23,...,0,0,0,0,0,0,0,0,1,0
99139,46.500000,0,1,3,8,<NA>,0,wtórny,<NA>,48,...,0,0,0,0,0,0,0,0,1,0


In [16]:
df=df.drop('dostepne_od',axis=1)


In [17]:
df['miejsce_parkingowe']=df['miejsce_parkingowe'].cat.add_categories('Brak').fillna('Brak')
#df['miejsce_parkingowe']=OrdinalEncoder().fit_transform(df['miejsce_parkingowe'])

In [18]:
cols=[col for col in df.columns if df[col].dtype=='category']
df[cols]=OrdinalEncoder().fit_transform(df[cols])

In [19]:
df=df.drop('opis',axis=1)

In [20]:
def preprocess_data(df,col_nan_count_treshold,row_nan_count_treshold,nan_value_impute_mode,byZScore,byIQR,iter,encode_mode,do_pca, drop_corr=True,corrRatio=0.75,pca_variance=0.95,standard_scaler=True,power_transformer=False):
    '''
    parametr - co robi - jakie może mieć wartości - wartość domyślna, jeżeli istnieje
    '''
    df=df.dropna(thresh=col_nan_count_treshold,axis=1)
    df=df.dropna(thresh=row_nan_count_treshold,axis=0)
    
    if nan_value_impute_mode=='KNN':    
        for col in [col for col in df.columns if df[col].isna().sum()>0]:
            print(col)
            if df[col].dtype == 'string[pyarrow]':
                df[col] =df[col].fillna('to_fill')
                mapping={cat:number for number,cat in enumerate(df[col].unique())}
                inversed_mapping = {v: k for k, v in mapping.items()}
                df[col]=df[col].map(mapping)
                imputer = KNNImputer(weights="distance", missing_values=mapping['to_fill'])
                df[[col,'latitude','longitude','ad_price']]=pd.DataFrame(imputer.fit_transform(df[[col,'latitude','longitude','ad_price']]),columns=[col,'latitude','longitude','ad_price'],index=df.index)
                df[col]=df[col].map(inversed_mapping)
            elif df[col].dtype != 'string[pyarrow]':
                df[col] =df[col].fillna(404)
                mapping={cat:number for number,cat in enumerate(df[col].unique())}
                inversed_mapping = {v: k for k, v in mapping.items()}
                df[col]=df[col].map(mapping)
                imputer = KNNImputer(weights="distance", missing_values=mapping[404])
                df[[col,'latitude','longitude','ad_price']]=pd.DataFrame(imputer.fit_transform(df[[col,'latitude','longitude','ad_price']]),columns=[col,'latitude','longitude','ad_price'],index=df.index)
                df[col]=df[col].map(inversed_mapping)
                
    df=out_outliers_and_bad_data(df,byZScore,byIQR,iter) #wyczyszczenie outlierów za pomocą funkcji zdefiniowanej wcześniej
    if drop_corr == True: #czy usunąć skorelowane kolumny
        df=dropCorr(df=df,corrRatio=corrRatio)
    df_features=df.drop(columns='ad_price') #nowa ramka danych bez ad_price
    df_num= df_features[df_features.describe().columns] #kolumny numeryczne
    df_num_ind,df_num_col=df_num.index,df_num.columns #indeksy i nazwy kolumn potrzebne później
    df_target=pd.DataFrame(df.Attrition,index=df_num_ind, columns=["Attrition"]) #ramka danych tylko z Attrition
    df_cat=df_features[df_features.describe(include='object').columns] #kolumny z danymi kategorialnymi
    if standard_scaler==True: #czy przeskalować
        sc=StandardScaler().fit(df_num)
        df_num=pd.DataFrame(sc.transform(df_num),index=df_num_ind,columns=df_num_col)
    if power_transformer == True:
        pt=PowerTransformer().fit(df_num)
        df_num=pd.DataFrame(pt.transform(df_num),index=df_num_ind,columns=df_num_col)
    if encode_mode == "OneHotEncoder": #czy skorzystać z OneHotEncodera
        df_encoded=pd.get_dummies(df_cat,drop_first=True,dtype=int)
        df_processed=pd.merge(df_num,df_encoded,left_index=True,right_index=True)
    elif encode_mode == "OrdinalEncoder": #czy też może z tego
        oe = OrdinalEncoder().fit(df_cat)
        df_cat_ind,df_cat_col=df_cat.index,df_cat.columns
        df_encoded=pd.DataFrame(oe.transform(df_cat),index=df_cat_ind,columns=df_cat_col)
        df_processed=pd.merge(df_num,df_encoded,left_index=True,right_index=True)

    X_train,X_test,y_train,y_test = train_test_split(df_processed,df_target.values.ravel(),test_size=0.2,random_state = 123)
    if do_pca == True: #redukcja wymiarowości z zachowaniem pca_variance wariancji
        pca = PCA()
        pca.fit(X_train)
        cumsum = np.cumsum(pca.explained_variance_ratio_)
        pca2=PCA(n_components=np.argmax(cumsum >= pca_variance) + 1)
        X_train=pca2.fit_transform(X_train)
        X_test=pca2.transform(X_test)
    return X_train,X_test,y_train,y_test

In [21]:
X_train,X_test,y_train,y_test = train_test_split(df.drop('ad_price',axis=1),df['ad_price'].values.ravel(),test_size=0.2,random_state = 123)

In [22]:
import mlflow
import mlflow.sklearn

def main():
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    # Initialize MLflow
    mlflow.set_experiment('House_pricing_test')

    models = []
    models.append(('LR', LinearRegression()))
    models.append(('MLP', MLPRegressor(random_state = 123)))
    models.append(('KNN', KNeighborsRegressor()))
    models.append(('DTC', DecisionTreeRegressor(random_state = 123)))
    models.append(('SVC', SVR()))
    models.append(('RF', RandomForestRegressor(random_state = 123)))
    models.append(('XGB', GradientBoostingRegressor(random_state = 123)))
    models.append(('BagR', BaggingRegressor(random_state = 123)))
    models.append(('ETR',ExtraTreesRegressor(random_state = 123)))
    models.append(('ABC',AdaBoostRegressor(random_state=123)))
    models.append(('SGD',SGDRegressor(random_state=123)))
    for name, model in models:
        with mlflow.start_run():
            cv_results = cross_val_score(model, X_train, y_train, cv = 5,n_jobs=-1)
            model.fit(X_train, y_train)
            mlflow.log_param('name',name)
            mlflow.log_metric('mean result',cv_results.mean())
            mlflow.sklearn.log_model(sk_model=model,artifact_path=f"model_{name}")



In [23]:
main()

2024/04/02 16:42:32 INFO mlflow.tracking.fluent: Experiment with name 'House_pricing_test' does not exist. Creating a new experiment.


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
/home/grzegorz/anaconda3/envs/tensorflow/lib/python3.11/site-packages/